In [10]:
import astropy
import astropy.units as u
import astroquery
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import quantity_support
from astropy.time import Time 
import os
from astropy.io.votable import parse_single_table
from astropy.time import Time
import pytest
from astropy.table import Table, vstack
import csv
import pandas as pd

from sympy import *
ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t = symbols('ra_0 gaia_ra pm_ra dec_0 gaia_dec pm_dec t')
init_printing(use_unicode=True)

First define ras and decs for both psr and gaia as functions of time, without including error

In [6]:
# psr
def psr_ra(ra_0, pm_ra, t):
    """ make sure to properly enter docstrings later on

    """
    ra_t = ra_0 + (pm_ra * t)
    return ra_t

def psr_dec(dec_0, pm_dec, t):
    """ make sure to properly enter docstrings later on

    """
    dec_t = dec_0 + (pm_dec * t)
    return dec_t  

# gaia -- for gaia we are taking the given epoch to be the epoch are searching in -- nothing requires propogating 
# through time so the ra and dec will just be variables and not functions

gaia_ra = 1
gaia_dec = 1

Now get difference between gaia and psr positions at the gaia epoch 

In [7]:
def diff_ra(ra_0, pm_ra, t):
    """ 
    """
    diff = psr_ra(ra_0, pm_ra, t) - gaia_ra
    return diff

def diff_dec(dec_0, pm_dec, t):
    """ 
    """
    diff = psr_ra(dec_0, pm_dec, t) - gaia_dec
    return diff

In [14]:
# mean difference in position
def r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t):
    return ((ra_0 - gaia_ra)**2 + (pm_ra * t)**2 + 2*(ra_0 - gaia_ra)*(pm_ra * t) + (dec_0 - gaia_dec)**2 + (pm_dec * t)**2 + 2*(dec_0 - gaia_dec)*(pm_dec * t))**(1/2)

In [16]:
dra_0sq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), ra_0)**2
ddec_0sq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), dec_0)**2
dgaia_rasq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), gaia_ra)**2
dgaia_decsq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), gaia_dec)**2
dpm_rasq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), pm_ra)**2
dpm_decsq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), pm_dec)**2

derivs = [dra_0sq, ddec_0sq, dgaia_rasq, dgaia_decsq, dpm_rasq, dpm_decsq]

In [ ]:
sig_ra0 # from atnf ra uncertainty
sig_dec0 # from atnf dec uncertainty
sig_gaiara # from gaia ra uncertainty
sig_gaiadec # from gaia dec uncertainty
sig_pmra # from atnf pmra uncertainty
sig_pmdec # from atnf pmdec uncertainty

sigs = [sig_ra0, sigdec0, sig_gaiara, sig_gaiadec, sig_pmra, sig_pmdec]

In [18]:
sig_r_sq = 0
count = 0
for i in derivs:
    sig_r_sq += i*(sigs[count]**2)
    count += 1 


NameError: name 'sigs' is not defined

In [ ]:
# this is just to get the error in the mean difference in position, so 

def error_propagation(ra_0, dec_0, pm_ra, pm_dec, raerr, decerr, pmraerr, pmdecerr, gaia_ra, gaia_dec, gaiaraerr, gaiadecerr):
    """
    
    """
    dra_0sq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), ra_0)**2
    ddec_0sq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), dec_0)**2
    dgaia_rasq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), gaia_ra)**2
    dgaia_decsq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), gaia_dec)**2
    dpm_rasq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), pm_ra)**2
    dpm_decsq = diff(r(ra_0, gaia_ra, pm_ra, dec_0, gaia_dec, pm_dec, t), pm_dec)**2

    derivs = [dra_0sq, ddec_0sq, dgaia_rasq, dgaia_decsq, dpm_rasq, dpm_decsq]

    sig_ra0 = raerr # from atnf ra uncertainty
    sig_dec0  = decerr# from atnf dec uncertainty
    sig_gaiara = gaiaraerr # from gaia ra uncertainty
    sig_gaiadec = gaiadecerr # from gaia dec uncertainty
    sig_pmra = pmraerr # from atnf pmra uncertainty
    sig_pmdec = pmdecerr # from atnf pmdec uncertainty

    sigs = [sig_ra0, sig_dec0, sig_gaiara, sig_gaiadec, sig_pmra, sig_pmdec]

    sig_r_sq = 0
    count = 0
    for i in derivs:
        sig_r_sq += i*(sigs[count]**2)
        count += 1 
    